This exercise is adapted from https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/


## Keras Tutorial Overview

There is not a lot of code required, but we are going to step over it slowly so that you will know how to create your own models in the future.

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

This Keras tutorial has a few requirements:

1. You have Python 2 or 3 installed and configured.
2. You have SciPy (including NumPy) installed and configured.
3. You have Keras and a backend (Theano or TensorFlow) installed and configured.

### 1. Load Data
The first step is to define the functions and classes we intend to use in this tutorial.

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

The imports required are listed below.

In [3]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense


import seaborn as sns

sns.set()


Now load the data from *file pima-indians-diabetes.data.csv* stored in Data folder (check Google Drive if unsure)

In [5]:
# load the dataset
dataset = loadtxt('.\pima-indians-diabetes.data.csv', delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

**Q:** Find out more about the dataset - such as how many columns, what kind of information does it provide etc.

In [6]:
import pandas as pd

df = pd.DataFrame(data=dataset)
# df.head()
df.describe()


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    float64
 1   1       768 non-null    float64
 2   2       768 non-null    float64
 3   3       768 non-null    float64
 4   4       768 non-null    float64
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    float64
 8   8       768 non-null    float64
dtypes: float64(9)
memory usage: 54.1 KB


### 2. Define Keras Model
Models in Keras are defined as a sequence of layers.

We create a **Sequential model** and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the **input_dim** argument and setting it to 8 for the 8 input variables.

*Question:* How do we know the number of layers and their types?

This is a very hard question. There are heuristics that we can use and often the best network structure is found through a process of trial and error experimentation (I explain more about this here). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the Dense class. We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the activation argument.

We will use the rectified linear unit activation function referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [41]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### 3. Compile Keras Model
Now that the model is defined, we can compile it.

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

We must specify the loss function to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

In this case, we will use cross entropy as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as **“binary_crossentropy“**. 

We will define the optimizer as the efficient stochastic gradient descent algorithm “adam“. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems. 

Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the metrics argument.

In [42]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Fit Keras Model
We have defined our model and compiled it ready for efficient computation.

Now it is time to execute the model on some data.

We can train or fit our model on our loaded data by calling the fit() function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch:** One pass through all of the rows in the training dataset.
* **Batch:** One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the batch_size argument.

For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called model convergence.

In [43]:
# fit the keras model on the dataset
model.fit(X,y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 1s 3ms/step - loss: 4.3964 - accuracy: 0.5768
Epoch 2/150
77/77 [==============================] - 0s 3ms/step - loss: 1.3446 - accuracy: 0.5651
Epoch 3/150
77/77 [==============================] - 0s 3ms/step - loss: 1.0017 - accuracy: 0.5664
Epoch 4/150
77/77 [==============================] - 0s 3ms/step - loss: 0.8499 - accuracy: 0.5833
Epoch 5/150
77/77 [==============================] - 0s 3ms/step - loss: 0.7395 - accuracy: 0.6133
Epoch 6/150
77/77 [==============================] - 0s 3ms/step - loss: 0.7330 - accuracy: 0.6250
Epoch 7/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6816 - accuracy: 0.6367
Epoch 8/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6356 - accuracy: 0.6602
Epoch 9/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6275 - accuracy: 0.6628
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6344 - accuracy: 0.6745
Epoch 11/

77/77 [==============================] - 0s 3ms/step - loss: 0.5260 - accuracy: 0.7161
Epoch 84/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5238 - accuracy: 0.7148
Epoch 85/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5498 - accuracy: 0.7161
Epoch 86/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5523 - accuracy: 0.7005
Epoch 87/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5513 - accuracy: 0.7122
Epoch 88/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5207 - accuracy: 0.7135
Epoch 89/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5524 - accuracy: 0.7031
Epoch 90/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5282 - accuracy: 0.7266
Epoch 91/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5540 - accuracy: 0.7227
Epoch 92/150
77/77 [==============================] - 0s 3ms/step - loss: 0.5397 - accuracy: 0.7031
Epoch 93/150


**Q:** change the value of Epoch - what differences will you expect/see?

In [44]:
# fit the keras model on the dataset
model.fit(X, y, epochs=300, batch_size=30)

Epoch 1/300
26/26 [==============================] - 0s 4ms/step - loss: 0.5043 - accuracy: 0.7370
Epoch 2/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4978 - accuracy: 0.7396
Epoch 3/300
26/26 [==============================] - 0s 6ms/step - loss: 0.4960 - accuracy: 0.7448
Epoch 4/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4902 - accuracy: 0.7487
Epoch 5/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4949 - accuracy: 0.7539
Epoch 6/300
26/26 [==============================] - 0s 5ms/step - loss: 0.4876 - accuracy: 0.7474
Epoch 7/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4882 - accuracy: 0.7461
Epoch 8/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4861 - accuracy: 0.7565
Epoch 9/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4865 - accuracy: 0.7474
Epoch 10/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4952 - accuracy: 0.7487
Epoch 11/

26/26 [==============================] - 0s 2ms/step - loss: 0.4815 - accuracy: 0.7526
Epoch 84/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4700 - accuracy: 0.7630
Epoch 85/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4879 - accuracy: 0.7552
Epoch 86/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4762 - accuracy: 0.7630
Epoch 87/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4733 - accuracy: 0.7643
Epoch 88/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4740 - accuracy: 0.7552
Epoch 89/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4687 - accuracy: 0.7669
Epoch 90/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4664 - accuracy: 0.7617
Epoch 91/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4716 - accuracy: 0.7617
Epoch 92/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4776 - accuracy: 0.7578
Epoch 93/300


26/26 [==============================] - 0s 3ms/step - loss: 0.4498 - accuracy: 0.7839
Epoch 165/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4596 - accuracy: 0.7734
Epoch 166/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4539 - accuracy: 0.7773
Epoch 167/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.7799
Epoch 168/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4737 - accuracy: 0.7695
Epoch 169/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4643 - accuracy: 0.7682
Epoch 170/300
26/26 [==============================] - 0s 6ms/step - loss: 0.4593 - accuracy: 0.7708
Epoch 171/300
26/26 [==============================] - 0s 7ms/step - loss: 0.4502 - accuracy: 0.7786
Epoch 172/300
26/26 [==============================] - 0s 4ms/step - loss: 0.4686 - accuracy: 0.7695
Epoch 173/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4770 - accuracy: 0.7604
Epoc

26/26 [==============================] - 0s 3ms/step - loss: 0.4443 - accuracy: 0.7852
Epoch 246/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4515 - accuracy: 0.7826
Epoch 247/300
26/26 [==============================] - 0s 3ms/step - loss: 0.4527 - accuracy: 0.7760
Epoch 248/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4644 - accuracy: 0.7826
Epoch 249/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4532 - accuracy: 0.7760
Epoch 250/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4476 - accuracy: 0.7786
Epoch 251/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4430 - accuracy: 0.7695
Epoch 252/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4624 - accuracy: 0.7643
Epoch 253/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4486 - accuracy: 0.7695
Epoch 254/300
26/26 [==============================] - 0s 2ms/step - loss: 0.4458 - accuracy: 0.7839
Epoc

### 5. Evaluate Keras Model
We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. The first will be the loss of the model on the dataset and the second will be the accuracy of the model on the dataset. We are only interested in reporting the accuracy, so we will ignore the loss value.

In [45]:
# evaluate the keras model
_, accuracy = model.evaluate(X,y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 2ms/step - loss: 0.4308 - accuracy: 0.7878
Accuracy: 78.78


### 7. Make Predictions
We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

Making predictions is as easy as calling the predict() function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [46]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions 
rounded = [round(x[0]) for x in predictions]


In [47]:
rounded

[1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,


**Q:** now redo predictions by using the function predict_classes(). Show the outcome of the first 10 cases (code provided)

In [48]:
import numpy as np

predictions = (model.predict(X) > 0.5).astype("int32")

for i in range(10):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 0 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 0 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


## Challenge 1:
Separate the training and test datasets (70% vs 30%) and rerun the evaluation

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=300, batch_size=30)

Epoch 1/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3576 - accuracy: 0.8380
Epoch 2/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3620 - accuracy: 0.8231
Epoch 3/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3626 - accuracy: 0.8250
Epoch 4/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3650 - accuracy: 0.8305
Epoch 5/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3574 - accuracy: 0.8250
Epoch 6/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3629 - accuracy: 0.8231
Epoch 7/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3558 - accuracy: 0.8305
Epoch 8/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3631 - accuracy: 0.8250
Epoch 9/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3713 - accuracy: 0.8250
Epoch 10/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3643 - accuracy: 0.8175
Epoch 11/

18/18 [==============================] - 0s 3ms/step - loss: 0.3689 - accuracy: 0.8175
Epoch 84/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3645 - accuracy: 0.8194
Epoch 85/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3580 - accuracy: 0.8250
Epoch 86/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3546 - accuracy: 0.8305
Epoch 87/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3618 - accuracy: 0.8194
Epoch 88/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3708 - accuracy: 0.8175
Epoch 89/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3702 - accuracy: 0.8250
Epoch 90/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3670 - accuracy: 0.8156
Epoch 91/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3584 - accuracy: 0.8212
Epoch 92/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3566 - accuracy: 0.8268
Epoch 93/300


18/18 [==============================] - 0s 2ms/step - loss: 0.3603 - accuracy: 0.8212
Epoch 165/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3658 - accuracy: 0.8175
Epoch 166/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3592 - accuracy: 0.8268
Epoch 167/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3631 - accuracy: 0.8287
Epoch 168/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3659 - accuracy: 0.8175
Epoch 169/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3535 - accuracy: 0.8250
Epoch 170/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3600 - accuracy: 0.8399
Epoch 171/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3555 - accuracy: 0.8268
Epoch 172/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3558 - accuracy: 0.8287
Epoch 173/300
18/18 [==============================] - 0s 2ms/step - loss: 0.3546 - accuracy: 0.8343
Epoc

18/18 [==============================] - 0s 3ms/step - loss: 0.3694 - accuracy: 0.8194
Epoch 246/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3623 - accuracy: 0.8212
Epoch 247/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3962 - accuracy: 0.8082
Epoch 248/300
18/18 [==============================] - 0s 4ms/step - loss: 0.3624 - accuracy: 0.8324
Epoch 249/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3556 - accuracy: 0.8250
Epoch 250/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3608 - accuracy: 0.8361
Epoch 251/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3952 - accuracy: 0.7970
Epoch 252/300
18/18 [==============================] - 0s 5ms/step - loss: 0.3742 - accuracy: 0.8380
Epoch 253/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3628 - accuracy: 0.8212
Epoch 254/300
18/18 [==============================] - 0s 3ms/step - loss: 0.3596 - accuracy: 0.8361
Epoc

In [30]:
# make probability predictions with the model
y_pred = model.predict(X_test)

y_pred_class = (y_pred > 0.5).astype("int32")

In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

         0.0       0.76      0.79      0.78       151
         1.0       0.58      0.54      0.56        80

    accuracy                           0.71       231
   macro avg       0.67      0.67      0.67       231
weighted avg       0.70      0.71      0.70       231



## Challenge 2:
How can you tune the model to improve the accuracy to be higher than what you have obtained?

In [38]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Users\tangkaiv\AppData\Local\Temp/ipykernel_12980/3020009828.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.717448 using {'batch_size': 10, 'epochs': 100}
0.647135 (0.021236) with: {'batch_size': 10, 'epochs': 10}
0.664062 (0.014616) with: {'batch_size': 10, 'epochs': 50}
0.717448 (0.021710) with: {'batch_size': 10, 'epochs': 100}
0.598958 (0.033804) with: {'batch_size': 20, 'epochs': 10}
0.673177 (0.023073) with: {'batch_size': 20, 'epochs': 50}
0.674479 (0.022402) with: {'batch_size': 20, 'epochs': 100}
0.579427 (0.017566) with: {'batch_size': 40, 'epochs': 10}
0.666667 (0.019488) with: {'batch_size': 40, 'epochs': 50}
0.697917 (0.014382) with: {'batch_size': 40, 'epochs': 100}
0.595052 (0.036272) with: {'batch_size': 60, 'epochs': 10}
0.656250 (0.050126) with: {'batch_size': 60, 'epochs': 50}
0.695312 (0.036782) with: {'batch_size': 60, 'epochs': 100}
0.502604 (0.025976) with: {'batch_size': 80, 'epochs': 10}
0.643229 (0.034104) with: {'batch_size': 80, 'epochs': 50}
0.634115 (0.043303) with: {'batch_size': 80, 'epochs': 100}
0.627604 (0.009744) with: {'batch_size': 100, 'epochs':

## Challenge 3:
How can you save the model and use it again for prediction directly without (re-)training?

In [53]:
model.save('\'')

SyntaxError: EOL while scanning string literal (Temp/ipykernel_12980/933960383.py, line 1)